In [34]:
## Call Libraries

import pandas as pd
from scipy import stats
import numpy as np
import random as rnd

In [42]:
## Load Data

skills   = pd.read_excel('Skills.xlsx')
green    = pd.read_excel('Green Occupations.xlsx')
desc     = pd.read_excel('Occupation Data.xlsx')
database = pd.read_csv('../../reshaped_data/database.csv')
roots    = pd.read_csv('code_root_names.csv')
auto     = pd.read_csv('automation_probability.csv')

skills.head()

,O*NET-SOC Code,Title,Element ID,Element Name,Scale ID,Scale Name,Data Value,N,Standard Error,Lower CI Bound,Upper CI Bound,Recommend Suppress,Not Relevant,Date,Domain Source
0,11-1011.00,Chief Executives,2.A.1.a,Reading Comprehension,IM,Importance,4.12,8,0.13,3.88,4.37,N,NaN,07/2014,Analyst
1,11-1011.00,Chief Executives,2.A.1.a,Reading Comprehension,LV,Level,4.75,8,0.16,4.43,5.07,N,N,07/2014,Analyst
2,11-1011.00,Chief Executives,2.A.1.b,Active Listening,IM,Importance,4.12,8,0.13,3.88,4.37,N,NaN,07/2014,Analyst
3,11-1011.00,Chief Executives,2.A.1.b,Active Listening,LV,Level,4.88,8,0.23,4.43,5.32,N,N,07/2014,Analyst
4,11-1011.00,Chief Executives,2.A.1.c,Writing,IM,Importance,4.00,8,0.00,4.00,4.00,N,NaN,07/2014,Analyst


In [41]:
## Helper Functions

def get_root(soc_code):
    return str(soc_code).split('-')[0]

def get_parent(soc_code):
    return str(soc_code).split('.')[0]

def get_id(soc_code):
    return str(soc_code).replace('-','').replace('.','')

def get_url(soc_code):
    slug = soc_code.replace('-','').split('.')[0]
#     return 'http://www.bls.gov/soc/2010/soc' + slug + '.htm'
    return 'https://www.mynextmove.org/profile/summary/' + soc_code

def get_category(soc_code):
    return str(soc_code).split('-')[0]

def get_distn(data,job):
    x = data[['Element Name','Data Value']][data['Id'] == job]
    return x.sort_values(['Element Name'])['Data Value'].tolist()

exp1 = np.exp(1)

def get_kld(v,w,base=exp1):
    #Kullback-Leibler Divergence
    return stats.entropy(v,w,base)

def get_edge(data,job1,job2):
    return get_kld(get_distn(data,job1),get_distn(data,job2))

rnd.seed(0)

In [37]:
## Clean Skills Data

cln         = skills[skills['Scale Name']=='Importance']
cln['Code'] = cln['O*NET-SOC Code']
cln['Id']   = cln['Title']

/usr/local/lib/python2.7/site-packages/IPython/kernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/site-packages/IPython/kernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [45]:
## Create Nodes File

nodes             = cln[['Id','Code']].drop_duplicates()

nodes['Url']      = nodes['Code'].apply(get_url)
nodes['Root']     = nodes['Code'].apply(get_root)
nodes['Par']      = nodes['Code'].apply(get_parent)
green['Code']     = green['O*NET-SOC Code']
desc['Code']      = desc['O*NET-SOC Code']
database['Code']  = database['O.NET.SOC.Code']
roots['Root']     = roots['Root'].apply(get_root)
auto['Par']       = auto['SOC Parent']
nodes             = nodes.merge(desc[['Code','Description']], how='left', on='Code')
nodes             = nodes.merge(green[['Code','Green Occupational Category']], how='left', on='Code')
nodes             = nodes.merge(database[['Code','salary_US','education_level_required_description']], how='left', on='Code')
nodes             = nodes.merge(roots[['Root','Category']], how='left', on='Root')
nodes             = nodes.merge(auto[['Par','Probability']], how='left', on='Par')

nodes.to_csv('nodes.csv')
nodes.head()

,Id,Code,Url,Root,Par,Description,Green Occupational Category,salary_US,education_level_required_description,Category,Probability
0,Chief Executives,11-1011.00,https://www.mynextmove.org/profile/summary/11-...,11,11-1011,Determine and formulate policies and provide o...,NaN,175110,Bachelor's Degree,Management Occupations,0.015
1,Chief Sustainability Officers,11-1011.03,https://www.mynextmove.org/profile/summary/11-...,11,11-1011,"Communicate and coordinate with management, sh...",Green New & Emerging,175110,Bachelor's Degree,Management Occupations,0.015
2,General and Operations Managers,11-1021.00,https://www.mynextmove.org/profile/summary/11-...,11,11-1021,"Plan, direct, or coordinate the operations of ...",Green Enhanced Skills,97730,Some College Courses,Management Occupations,0.160
3,Advertising and Promotions Managers,11-2011.00,https://www.mynextmove.org/profile/summary/11-...,11,11-2011,"Plan, direct, or coordinate advertising polici...",NaN,95890,Associate's Degree (or other 2-year degree),Management Occupations,0.039
4,Marketing Managers,11-2021.00,https://www.mynextmove.org/profile/summary/11-...,11,11-2021,"Plan, direct, or coordinate marketing policies...",Green Enhanced Skills,128750,Bachelor's Degree,Management Occupations,0.014


In [ ]:
## Calculate all edges

jobs_list = nodes['Id'].tolist()
jobs_list.sort()
edges_all = []
for i in range(len(jobs_list)):
    print i
    job1 = jobs_list[i]
    for j in range(i+1,len(jobs_list)):
        job2 = jobs_list[j]
        edges_all = edges_all + [[str(job1),str(job2),get_edge(cln,job1,job2)]]
edges_all = pd.DataFrame(edges_all, columns = {'Source','Target','Weight'})
edges_all.head()

In [11]:
## Create Edges File

pctile    = 100 * len(jobs_list) * 5 / float(len(edges_all))
threshold = np.percentile(edges_all['Weight'],pctile)
edges     = edges_all[edges_all['Weight'] <= threshold]
edges.to_csv('edges.csv')
print len(jobs_list), len(edges_all), pctile, threshold, len(edges)
edges.head()

954 454581 1.04931794334 0.00719966667374 4770


,Unnamed: 0,Source,Target,Weight
5,5,11-1011.00,11-2031.00,0.006473
15,15,11-1011.00,11-3071.01,0.006828
19,19,11-1011.00,11-3121.00,0.006084
42,42,11-1011.00,11-9151.00,0.006443
121,121,11-1011.00,15-1199.09,0.006457
